In [29]:
from podcast_ad_skipper.google_cloud import *
from podcast_ad_skipper.params import *
from podcast_ad_skipper.data_preparation import *
import pandas as pd
import json
from podcast_ad_skipper.main import get_processed_training_data

In [30]:
storage_client = auth_gc_storage()


Running in local environment.
Authenticated successfully with GCS! ✅


In [36]:
bucket_name = BUCKET_NAME
prefixes=GCP_PREFIXES[0]
chunk_size=5
table_name='test8'

In [32]:
file_list = retrieve_files_in_folder(storage_client, bucket_name, prefixes)

In [33]:
chunks_list = make_chunks(file_list, chunk_size)

In [35]:
%%time

bq_client = auth_gc_bigquery()

for chunk in chunks_list[:3]:

    features = get_features_model(chunk)

    table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.test8"

    rows_to_insert = [
        {
            "spectrogram": json.dumps(features[0][i].tolist()),
            "labels": features[1][i],
            "seconds": features[2][i],
            "duration": features[3][i],
            "podcast_names": features[4][i],
        } for i in range(len(features[0]))
    ]
    insert_data_to_bq(rows_to_insert, bq_client, table_id, chunk_size)
    # Time out error again, with both [:1] and [:10]
    # Move spectrogram code to model pipeline

Running in local environment.
Authenticated successfully with BigQuery! ✅
Processing files: total 5
Batch of 5 rows has been added.
Processing files: total 5
Batch of 5 rows has been added.
Processing files: total 5
Batch of 5 rows has been added.
CPU times: user 978 ms, sys: 236 ms, total: 1.21 s
Wall time: 23.3 s


In [16]:
table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.test8"

In [17]:
output= get_output_query_bigquery(bq_client, table_id)


Query returned 10 results


In [18]:
processed_output = get_bq_processed_data(output)

In [23]:
processed_output[0][0].shape

(128, 157, 3)

In [38]:
get_processed_training_data(prefixes, table_name, chunk_size)

Running in local environment.
Authenticated successfully with GCS! ✅
Running in local environment.
Authenticated successfully with BigQuery! ✅
Batch of 5 rows has been added.
Number of processed files:5
Batch of 5 rows has been added.
Number of processed files:10


KeyboardInterrupt: 